# Reranking for Enhanced RAG Systems

### Reranking의 주요 개념 및 단계

**1. 초기 검색 (First-pass Retrieval):**
벡터 임베딩 기반의 기본 유사도 검색(Cosine similarity 등)을 사용하여 쿼리와 유사한 문서 청크들을 빠르게 검색한다.
이 단계는 속도는 빠르지만 정확도는 상대적으로 낮을 수 있다.
예: FAISS, Elasticsearch, Weaviate 등을 활용하여 top-20 청크를 검색하다.

**2. 문서 채점 (Scoring):**
검색된 각 문서(또는 청크)에 대해 쿼리와의 관련성을 정밀하게 평가한다.
단순 임베딩 유사도 대신, Cross-Encoder 또는 LLM 기반 점수 평가 모델을 사용하여 정교한 채점을 수행한다.
Cross-Encoder: 입력으로 (query, document) 쌍을 받아 직접 관련성 점수를 출력한다.
예: bge-reranker, ms-marco-TinyBERT, OpenAI function-calling 기반 평가 모델 등이 있다.

**3. 재정렬 (Reranking):**
점수에 따라 문서들을 내림차순으로 정렬한다.
이 과정에서 덜 관련된 청크는 하위로 밀려나며, 최종 응답에 포함되지 않을 수 있다.

**4. 선택 (Selection):**
상위 N개의 청크(예: top-3 또는 top-5)를 최종 선택하여 LLM에 입력한다.
선택된 문서들은 더 높은 신뢰도와 일관성을 기반으로 응답 생성에 사용된다.


### 예시 흐름 요약

[사용자 쿼리] → [초기 검색 (top-20)] → [재랭크 모델로 관련성 점수 평가] → [점수 기반 정렬] → [top-3 청크 선택] → [LLM에 전달하여 응답 생성]


## Setting Up the Environment

In [1]:
import fitz
import numpy as np
import json
import re

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from a PDF File

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일의 경로입니다.

    Returns:
        str: PDF에서 추출한 텍스트.
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 빈 문자열 초기화

    # PDF의 각 페이지를 반복하며 텍스트를 추출
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]                  # 페이지 객체 가져오기
        text = page.get_text("text")            # 텍스트 추출
        all_text += text                        # 텍스트 누적

    return all_text  # 전체 텍스트 반환

## Chunking the Extracted Text

In [4]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 겹치는 n개의 문자 세그먼트로 청크합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간에 겹치는 문자 수입니다.

    Returns:
    List[str]: 텍스트 청크의 목록입니다.
    """
    chunks = []  # 청크를 저장할 빈 목록을 초기화합니다.

    # 단계 크기 (n - 겹침)로 텍스트를 반복합니다.
    for i in range(0, len(text), n - overlap):
        # 청크 목록에 인덱스 i에서 i + n까지의 텍스트 청크를 추가합니다.
        chunks.append(text[i:i + n])

    return chunks  # 텍스트 청크 목록을 반환합니다.

## Building a Simple VectorStore

In [5]:
class SimpleVectorStore:
    """
    NumPy를 사용한 간단한 벡터 스토어 구현.
    """
    def __init__(self):
        """
        Initialize the vector store.
        """
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 스토어에 항목을 추가합니다.
        Args:
        text (str): 원본 텍스트입니다.
        embedding (List[float]): 임베딩 벡터입니다.
        metadata (dict, optional): 추가 메타데이터.
        """
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def similarity_search(self, query_embedding, k=5):
        """
        쿼리 임베딩과 가장 유사한 항목을 찾습니다.

        Args:
        query_embedding  (List[float]): 쿼리 임베딩 벡터.
        k (int): 반환할 결과의 개수.

        Returns:
        List[Dict]: 텍스트와 메타데이터가 가장 유사한 상위 k개 항목입니다.
        """
        if not self.vectors:
            return []
        
        # 쿼리 임베딩을 numpy 배열로 변환하기
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 사용하여 유사도 계산하기
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 유사도 기준 정렬(내림차순)
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })
        
        return results

## Creating Embeddings

In [6]:
def create_embeddings(text, model="text-embedding-3-small"):
    """
    지정된 OpenAI 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
    text (str): 임베딩을 생성할 입력 텍스트입니다.
    모델 (str): 임베딩을 만드는 데 사용할 모델입니다.

    Returns:
    List[float]: 임베딩 벡터입니다.
    """
    # 문자열 입력을 목록으로 변환하여 문자열 입력과 목록 입력을 모두 처리합니다.
    input_text = text if isinstance(text, list) else [text]

    # 지정된 모델을 사용하여 입력 텍스트에 대한 임베딩을 생성합니다.
    response = client.embeddings.create(
        model=model,
        input=input_text
    )

    # 입력이 문자열인 경우, 첫 번째 임베딩만 반환합니다.
    if isinstance(text, str):
        return response.data[0].embedding

    # 그렇지 않으면 모든 임베딩을 벡터 목록으로 반환합니다.
    return [item.embedding for item in response.data]

## Document Processing Pipeline

In [7]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    RAG용 문서를 처리합니다.

    Args:
        pdf_path (str): PDF 파일의 경로입니다.
        chunk_size (int): 각 청크의 크기(문자 단위).
        chunk_overlap (int): 청크 간 중첩되는 문자 수.

    Returns:
        SimpleVectorStore: 문서 청크와 해당 임베딩이 포함된 벡터 저장소.
    """
    print("PDF에서 텍스트를 추출합니다...")
    extracted_text = extract_text_from_pdf(pdf_path)

    print("텍스트를 청크 단위로 분할합니다...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"{len(chunks)}개의 텍스트 청크가 생성되었습니다.")

    print("각 청크에 대한 임베딩을 생성합니다...")
    # 효율성을 위해 모든 청크에 대해 한 번에 임베딩 생성
    chunk_embeddings = create_embeddings(chunks)

    # 벡터 저장소 생성
    store = SimpleVectorStore()

    # 벡터 저장소에 청크 추가
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={"index": i, "source": pdf_path}
        )

    print(f"총 {len(chunks)}개의 청크가 벡터 저장소에 추가되었습니다.")
    return store

## Implementing LLM-based Reranking

In [8]:
def rerank_with_llm(query, results, top_n=3, model="gpt-4o-mini"):
    """
    LLM을 활용하여 검색 결과를 관련성 기준으로 재정렬합니다.

    Args:
        query (str): 사용자의 검색 질의.
        results (List[Dict]): 초기 검색 결과 목록.
        top_n (int): 재정렬 후 상위에 올 결과 수.
        model (str): 관련성 평가에 사용할 LLM 모델 이름.

    Returns:
        List[Dict]: 관련성 기준으로 재정렬된 상위 결과 리스트.
    """
    print(f"{len(results)}개의 문서를 LLM을 이용해 재정렬합니다...")

    scored_results = []  # 관련성 점수를 포함한 결과 저장용 리스트

    # 시스템 프롬프트 정의: 관련성 평가 기준 안내
    system_prompt = """너는 검색어에 대한 문서 관련성을 평가하는 전문가입니다.
주어진 쿼리에 얼마나 잘 답변하는지를 기준으로 문서를 0~10 점수로 평가하세요.

평가기준:
- 0~2점: 전혀 관련 없음
- 3~5점: 일부 관련 있으나 직접적인 답변은 아님
- 6~8점: 관련 있으며 부분적으로 답변함
- 9~10점: 매우 관련 있으며 직접적인 답변을 포함함

단일 숫자(0~10)만 응답하세요. 그 외 텍스트는 포함하지 마세요."""

    # 각 검색 결과에 대해 LLM을 사용한 관련성 평가 수행
    for i, result in enumerate(results):
        if i % 5 == 0:
            print(f"{i+1}/{len(results)} 문서 평가 중...")

        user_prompt = f"""Query: {query}

Document:
{result['text']}

이 문서가 쿼리에 얼마나 관련 있는지 0~10 사이의 점수로 평가하세요:"""

        # 모델 호출
        response = client.chat.completions.create(
            model=model,
            temperature=0,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )

        # 점수 추출
        score_text = response.choices[0].message.content.strip()
        score_match = re.search(r'\b(10|[0-9])\b', score_text)

        if score_match:
            score = float(score_match.group(1))
        else:
            print(f"점수 추출 실패: '{score_text}' → similarity 점수 사용")
            score = result["similarity"] * 10

        scored_results.append({
            "text": result["text"],
            "metadata": result["metadata"],
            "similarity": result["similarity"],
            "relevance_score": score
        })

    # 관련성 점수를 기준으로 정렬
    reranked_results = sorted(scored_results, key=lambda x: x["relevance_score"], reverse=True)

    # 상위 top_n개만 반환
    return reranked_results[:top_n]

## Simple Keyword-based Reranking

In [9]:
def rerank_with_keywords(query, results, top_n=3):
    """
    키워드 매칭 및 위치 기반의 간단한 재정렬 방식
    
    Args:
        query (str): 사용자의 검색 질의
        results (List[Dict]): 초기 검색 결과 목록
        top_n (int): 재정렬 후 반환할 결과 개수
        
    Returns:
        List[Dict]: 재정렬된 결과 목록
    """
    # 질의에서 중요 키워드를 추출 (길이가 3자 초과하는 단어만 선택)
    keywords = [word.lower() for word in query.split() if len(word) > 3]

    scored_results = []  # 점수를 매긴 결과를 저장할 리스트

    for result in results:
        document_text = result["text"].lower()  # 문서 내용을 소문자로 변환

        # 기본 점수는 벡터 유사도에서 시작 (0.5 가중치)
        base_score = result["similarity"] * 0.5

        keyword_score = 0  # 키워드 관련 점수 초기화
        for keyword in keywords:
            if keyword in document_text:
                # 키워드가 포함되어 있으면 0.1점 추가
                keyword_score += 0.1

                # 키워드가 문서 초반(1/4 지점 이내)에 있으면 추가로 0.1점
                first_position = document_text.find(keyword)
                if first_position < len(document_text) / 4:
                    keyword_score += 0.1

                # 키워드 등장 빈도에 따라 점수 추가 (최대 0.2점까지)
                frequency = document_text.count(keyword)
                keyword_score += min(0.05 * frequency, 0.2)

        # 최종 점수는 기본 점수 + 키워드 점수
        final_score = base_score + keyword_score

        # 점수 포함 결과를 리스트에 추가
        scored_results.append({
            "text": result["text"],
            "metadata": result["metadata"],
            "similarity": result["similarity"],
            "relevance_score": final_score
        })

    # 관련성 점수를 기준으로 내림차순 정렬
    reranked_results = sorted(scored_results, key=lambda x: x["relevance_score"], reverse=True)

    # 상위 top_n 개 결과만 반환
    return reranked_results[:top_n]

## Response Generation

In [10]:
def generate_response(query, context, model="gpt-4o-mini"):
    """
    주어진 질의(query)와 문맥(context)을 바탕으로 응답을 생성합니다.
    
    Args:
        query (str): 사용자의 질문
        context (str): 검색된 문맥 정보
        model (str): 응답 생성을 위해 사용할 LLM 모델 이름
        
    Returns:
        str: 생성된 응답 문자열
    """
    # AI의 응답 방식에 대한 지침을 담은 시스템 프롬프트 정의
    system_prompt = (
        "당신은 유용한 AI 비서입니다. "
        "제공된 컨텍스트에 따라서만 사용자의 질문에 답변하세요. "
        "문맥에서 답을 찾을 수 없는 경우 정보가 충분하지 않다고 말하세요."
    )

    # 사용자 프롬프트 생성: 문맥 + 질문 조합
    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        위의 문맥에만 근거하여 포괄적인 답변을 제공해 주세요.
    """

    # 지정된 모델을 사용하여 응답 생성
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # 생성된 응답 내용을 반환
    return response.choices[0].message.content

## Full RAG Pipeline with Reranking

In [11]:
def rag_with_reranking(query, vector_store, reranking_method="llm", top_n=3, model="gpt-4o-mini"):
    """
    재정렬 기능을 포함한 RAG 파이프라인 전체 흐름
    
    Args:
        query (str): 사용자의 질의
        vector_store (SimpleVectorStore): 벡터 검색이 가능한 저장소
        reranking_method (str): 재정렬 방식 ('llm' 또는 'keywords')
        top_n (int): 재정렬 후 사용할 상위 문서 개수
        model (str): 응답 생성을 위한 LLM 모델 이름
        
    Returns:
        Dict: 질의, 문맥, 응답 및 중간 결과가 포함된 딕셔너리
    """
    # 질의 임베딩 생성
    query_embedding = create_embeddings(query)

    # 초기 검색 수행 (재정렬을 위해 충분히 많이 검색, 예: 10개)
    initial_results = vector_store.similarity_search(query_embedding, k=10)

    # 재정렬 수행
    if reranking_method == "llm":
        # LLM을 활용한 관련성 기반 재정렬
        reranked_results = rerank_with_llm(query, initial_results, top_n=top_n)
    elif reranking_method == "keywords":
        # 키워드 기반 간단한 재정렬
        reranked_results = rerank_with_keywords(query, initial_results, top_n=top_n)
    else:
        # 재정렬 없이 초기 검색 결과 상위 n개 사용
        reranked_results = initial_results[:top_n]

    # 재정렬된 결과에서 문맥(context) 구성
    context = "\n\n===\n\n".join([result["text"] for result in reranked_results])

    # 문맥을 기반으로 응답 생성
    response = generate_response(query, context, model)

    # 최종 결과 반환 (디버깅이나 로그용으로 중간 값도 포함)
    return {
        "query": query,
        "reranking_method": reranking_method,
        "initial_results": initial_results[:top_n],
        "reranked_results": reranked_results,
        "context": context,
        "response": response
    }

## Evaluating Reranking Quality

In [12]:
# JSON 파일에서 검증용 데이터 로드
with open('../dataset/validation.json') as f:
    data = json.load(f)

# 검증 데이터에서 첫 번째 질문 추출
query = data[0]['question']

# 검증 데이터에서 해당 질문의 정답(참조 답변) 추출
reference_answer = data[0]['ideal_answer']

# 사용할 PDF 파일 경로 정의
pdf_path = "../dataset/AI_Understanding.pdf"

In [13]:
# PDF 문서를 처리하여 벡터 저장소 생성
vector_store = process_document(pdf_path)

# 테스트용 질의 정의
query = "AI는 우리의 생활과 업무 방식을 변화시킬 잠재력을 가지고 있을까요?"

# 다양한 검색 및 재정렬 방식 비교
print("=== 검색 및 재정렬 방식 비교 ===")

# 1. 기본 검색 (재정렬 없이)
print("\n--- [1] STANDARD RETRIEVAL ---")
standard_results = rag_with_reranking(query, vector_store, reranking_method="none")
print(f"\n[질문]\n{query}")
print(f"\n[응답]\n{standard_results['response']}")

# 2. LLM 기반 재정렬
print("\n--- [2] LLM-BASED RERANKING ---")
llm_results = rag_with_reranking(query, vector_store, reranking_method="llm")
print(f"\n[질문]\n{query}")
print(f"\n[응답]\n{llm_results['response']}")

# 3. 키워드 기반 재정렬
print("\n--- [3] KEYWORD-BASED RERANKING ---")
keyword_results = rag_with_reranking(query, vector_store, reranking_method="keywords")
print(f"\n[질문]\n{query}")
print(f"\n[응답]\n{keyword_results['response']}")

PDF에서 텍스트를 추출합니다...
텍스트를 청크 단위로 분할합니다...
21개의 텍스트 청크가 생성되었습니다.
각 청크에 대한 임베딩을 생성합니다...
총 21개의 청크가 벡터 저장소에 추가되었습니다.
=== 검색 및 재정렬 방식 비교 ===

--- [1] STANDARD RETRIEVAL ---

[질문]
AI는 우리의 생활과 업무 방식을 변화시킬 잠재력을 가지고 있을까요?

[응답]
네, AI는 우리의 생활과 업무 방식을 변화시킬 잠재력을 가지고 있습니다. AI는 비즈니스 운영을 혁신하고, 고객 관계 관리(CRM)를 향상시키며, 공급망 관리와 인적 자원(HR) 분야에서도 효율성을 높이는 데 기여하고 있습니다. 또한, AI는 마케팅 및 영업 활동을 개선하고, 금융 서비스에서 사기 탐지와 위험 관리에 활용되고 있습니다.

AI의 발전은 일자리 대체에 대한 우려를 불러일으키기도 하지만, 동시에 새로운 기회를 창출하고 기존 역할을 변화시키는 긍정적인 측면도 있습니다. 인간과 AI의 협업이 활발해지면서, AI 도구는 인간의 역량을 강화하고 의사 결정을 지원하는 인사이트를 제공할 수 있습니다. 

또한, AI는 창의성과 혁신을 위한 도구로도 활용되며, 예술, 음악, 문학 등 다양한 분야에서 새로운 가능성을 제시하고 있습니다. 이러한 변화는 교육 및 인력 개발, 윤리적 고려 사항과 같은 다양한 측면에서의 접근이 필요하며, 인간 중심의 접근 방식을 통해 AI의 잠재력을 최대한 활용할 수 있습니다.

--- [2] LLM-BASED RERANKING ---
10개의 문서를 LLM을 이용해 재정렬합니다...
1/10 문서 평가 중...
6/10 문서 평가 중...

[질문]
AI는 우리의 생활과 업무 방식을 변화시킬 잠재력을 가지고 있을까요?

[응답]
네, AI는 우리의 생활과 업무 방식을 변화시킬 잠재력을 가지고 있습니다. AI는 다양한 산업 분야에서 비즈니스 운영을 혁신하여 효율성을 향상시키고, 비용을 절감하며, 의사 결정을 개선하는 데 기여하고 있습니다. 예를 들

In [14]:
def evaluate_reranking(query, standard_results, reranked_results, reference_answer=None):
    """
    기본 검색 결과와 재정렬된 결과를 비교 평가합니다.
    
    Args:
        query (str): 사용자 질의
        standard_results (Dict): 기본 검색 방식의 결과
        reranked_results (Dict): 재정렬된 검색 방식의 결과
        reference_answer (str, optional): 기준이 되는 정답 (선택사항)
        
    Returns:
        str: 평가 결과 텍스트
    """
    # AI 평가자에게 역할을 부여하는 시스템 프롬프트 정의
    system_prompt = """귀하는 RAG 시스템의 전문 평가자입니다.
    두 가지 다른 검색 방법에서 검색된 컨텍스트와 응답을 비교하세요.
    어떤 것이 더 나은 컨텍스트와 더 정확하고 포괄적인 답변을 제공하는지 평가하세요."""

    # 비교를 위한 텍스트 구성 (문맥은 1000자까지만 표시)
    comparison_text = f"""Query: {query}

Standard Retrieval Context:
{standard_results['context'][:1000]}... [truncated]

Standard Retrieval Answer:
{standard_results['response']}

Reranked Retrieval Context:
{reranked_results['context'][:1000]}... [truncated]

Reranked Retrieval Answer:
{reranked_results['response']}"""

    # 참조 정답(reference answer)이 있다면 비교 텍스트에 포함
    if reference_answer:
        comparison_text += f"""
        
Reference Answer:
{reference_answer}"""

    # 사용자 프롬프트 구성: 어떤 방식이 더 나은지 평가 요청
    user_prompt = f"""
{comparison_text}

제공된 검색 방법을 평가해 주세요:
1. 더 관련성 높은 컨텍스트
2. 더 정확한 답변
3. 보다 포괄적인 답변
4. 전반적인 성능 향상

구체적인 예시와 함께 자세한 분석을 제공하세요.
"""

    # AI 모델을 사용하여 평가 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # 평가 결과 반환
    return response.choices[0].message.content

In [15]:
# LLM 기반 재정렬 결과가 기본 검색 결과보다 더 나은지 평가
evaluation = evaluate_reranking(
    query=query,  # 사용자 질의
    standard_results=standard_results,  # 기본 검색 결과
    reranked_results=llm_results,  # LLM 기반 재정렬 결과
    reference_answer=reference_answer  # 참조 정답 (비교 기준)
)

# 평가 결과 출력
print("\n***평가 결과***")
print(evaluation)


***평가 결과***
### 평가 분석

1. **더 관련성 높은 컨텍스트**:
   - **Standard Retrieval Context**: 이 컨텍스트는 AI의 글로벌 협업, 교육 및 인력 개발, 인간 중심의 접근 방식에 대한 내용을 포함하고 있습니다. 그러나 AI가 우리의 생활과 업무 방식을 변화시키는 구체적인 예시나 산업별 적용에 대한 정보는 부족합니다.
   - **Reranked Retrieval Context**: 이 컨텍스트는 AI의 다양한 산업 분야에서의 구체적인 적용 사례(비즈니스 운영 혁신, 고객 관계 관리, 공급망 관리 등)를 포함하고 있어, 질문에 대한 직접적인 관련성이 높습니다. AI의 실제 활용 사례를 통해 독자가 AI의 영향을 더 잘 이해할 수 있도록 돕습니다.

2. **더 정확한 답변**:
   - **Standard Retrieval Answer**: 이 답변은 AI의 잠재력에 대한 일반적인 설명을 제공하지만, 구체적인 산업별 예시가 부족하여 다소 모호하게 느껴질 수 있습니다.
   - **Reranked Retrieval Answer**: 이 답변은 AI의 다양한 적용 분야를 구체적으로 언급하며, 각 분야에서의 변화와 이점에 대해 명확하게 설명하고 있습니다. 예를 들어, 고객 관계 관리에서의 개인화된 경험 제공, 공급망 관리에서의 최적화 등 구체적인 사례를 통해 AI의 영향을 정확하게 전달하고 있습니다.

3. **보다 포괄적인 답변**:
   - **Standard Retrieval Answer**: AI의 긍정적인 측면과 일자리 대체에 대한 우려를 언급하지만, 구체적인 산업별 변화나 AI의 다양한 활용 가능성에 대한 설명이 부족합니다.
   - **Reranked Retrieval Answer**: 이 답변은 AI의 긍정적인 측면과 함께 일자리 대체에 대한 우려를 다루며, 재교육 및 업스킬링의 중요성을 강조합니다. 또한, AI가 창의성과 혁신을 촉진할 수 있는 도구로 자리잡을 수 있다는 점을 언급하여 포괄적인 시각을